# 1

In [ ]:
#pip install torch pandas numpy

# 2

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Đọc dữ liệu từ file CSV
file_path = 'UNSW_NB15_training-set.csv'
df = pd.read_csv(file_path)

# Hiển thị kiểu dữ liệu của các cột
print(df.dtypes)

# Lựa chọn các cột số và cột không phải là số
numeric_columns = df.select_dtypes(include=['number']).columns
categorical_columns = df.select_dtypes(exclude=['number']).columns

# Chuyển đổi các cột không phải là số sang mã số
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Chuẩn hóa dữ liệu cho các cột số
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# Kiểm tra lại dữ liệu đã chuẩn hóa
print(df.head())

# Dữ liệu đã được chuẩn hóa và chuyển đổi sẵn sàng để sử dụng cho mô hình GAN
data = df.values

id                     int64
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean                  int64
trans_depth            int64
response_body_len      int64
ct_srv_src             int64
ct_state_ttl           int64
ct_dst_ltm             int64
ct_src_dport_l

# 3

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
        )
    
    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        return self.model(x)

# 4

In [5]:
# Định nghĩa các siêu tham số
input_dim = 100  # Kích thước của vector ngẫu nhiên đầu vào cho Generator
output_dim = data.shape[1]  # Kích thước của dữ liệu đầu ra
batch_size = 64
num_epochs = 1000
lr = 0.0002

# Tạo các mô hình Generator và Discriminator
generator = Generator(input_dim, output_dim)
discriminator = Discriminator(output_dim)

# Định nghĩa các hàm mất mát và bộ tối ưu hóa
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr)
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr)

# Huấn luyện mô hình GAN
for epoch in range(num_epochs):
    for i in range(0, data.shape[0], batch_size):
        # Tạo batch dữ liệu thật
        real_data = torch.tensor(data[i:i+batch_size], dtype=torch.float32)
        real_labels = torch.ones(real_data.size(0), 1)
        
        # Tạo batch dữ liệu giả
        noise = torch.randn(real_data.size(0), input_dim)
        fake_data = generator(noise)
        fake_labels = torch.zeros(fake_data.size(0), 1)
        
        # Huấn luyện Discriminator
        optimizer_d.zero_grad()
        outputs_real = discriminator(real_data)
        loss_real = criterion(outputs_real, real_labels)
        outputs_fake = discriminator(fake_data.detach())
        loss_fake = criterion(outputs_fake, fake_labels)
        loss_d = loss_real + loss_fake
        loss_d.backward()
        optimizer_d.step()
        
        # Huấn luyện Generator
        optimizer_g.zero_grad()
        outputs_fake = discriminator(fake_data)
        loss_g = criterion(outputs_fake, real_labels)
        loss_g.backward()
        optimizer_g.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss D: {loss_d.item()}, Loss G: {loss_g.item()}')

Epoch [100/1000], Loss D: 0.06830091774463654, Loss G: 5.1047821044921875
Epoch [200/1000], Loss D: 0.4101049304008484, Loss G: 31.186555862426758
Epoch [300/1000], Loss D: 0.0, Loss G: 100.0
Epoch [400/1000], Loss D: 0.0, Loss G: 100.0
Epoch [500/1000], Loss D: 0.0, Loss G: 100.0
Epoch [600/1000], Loss D: 0.0, Loss G: 100.0
Epoch [700/1000], Loss D: 0.0, Loss G: 100.0
Epoch [800/1000], Loss D: 0.0, Loss G: 100.0
Epoch [900/1000], Loss D: 0.0, Loss G: 100.0
Epoch [1000/1000], Loss D: 0.0, Loss G: 100.0


# 5

In [ ]:
# Sinh khoảng 15 triệu mẫu dữ liệu giả
num_samples = 15000000  # Số lượng mẫu dữ liệu giả cần sinh
batch_size = 10000  # Kích thước mỗi batch để sinh dữ liệu giả

fake_data_list = []
for _ in range(num_samples // batch_size):
    noise = torch.randn(batch_size, input_dim)
    fake_data_batch = generator(noise).detach().numpy()
    fake_data_list.append(fake_data_batch)

# Kết hợp tất cả các batch thành một mảng numpy lớn
fake_data = np.vstack(fake_data_list)

# Kiểm tra số lượng cột của dữ liệu giả trước khi chuyển đổi
assert fake_data.shape[1] == scaler.mean_.shape[0], "Số lượng cột của dữ liệu giả không khớp với dữ liệu gốc."

# Chuyển đổi dữ liệu giả về dạng ban đầu
fake_data = scaler.inverse_transform(fake_data)

# Tạo DataFrame từ dữ liệu giả
fake_df = pd.DataFrame(fake_data, columns=df.columns)

# Lưu toàn bộ dữ liệu giả vào một tệp CSV
fake_df.to_csv('generated_data_15m_samples.csv', index=False)
print('Generated data saved to generated_data_15m_samples.csv')